In [3]:
import geopandas as gpd
from pykml import parser
from shapely.geometry import Point, LineString, Polygon
from shapely.ops import unary_union

def parse_kml_coordinates(coordinates_text, geom_type):
    coords = coordinates_text.strip().split()
    parsed_coords = [tuple(map(float, coord.split(','))) for coord in coords]
    
    if geom_type == 'Point':
        return Point(parsed_coords[0])
    elif geom_type == 'LineString':
        return LineString(parsed_coords)
    elif geom_type == 'Polygon':
        return Polygon(parsed_coords)

# Load the KML file
kml_file = r"C:\Users\91956\Desktop\Auroville Consulting\SHUBAM\India_TN_Greenfield_FABDEM\India_TN_Greenfield_FABDEM\kmls\15GWh_18h_summary_0.kml"

with open(kml_file) as f:
    root = parser.parse(f).getroot()

# Extract Points, Lines, and Polygons with attributes
data_points = []
data_lines = []
data_polygons = []

for placemark in root.Document.findall('.//{http://www.opengis.net/kml/2.2}Placemark'):
    name = placemark.find('.//{http://www.opengis.net/kml/2.2}name')
    name = name.text if name is not None else None
    
    for point in placemark.findall('.//{http://www.opengis.net/kml/2.2}Point'):
        coordinates_text = point.coordinates.text
        geometry = parse_kml_coordinates(coordinates_text, 'Point')
        data_points.append({'name': name, 'geometry': geometry})
        
    for linestring in placemark.findall('.//{http://www.opengis.net/kml/2.2}LineString'):
        coordinates_text = linestring.coordinates.text
        geometry = parse_kml_coordinates(coordinates_text, 'LineString')
        data_lines.append({'name': name, 'geometry': geometry})
        
    for polygon in placemark.findall('.//{http://www.opengis.net/kml/2.2}Polygon'):
        coordinates_text = polygon.outerBoundaryIs.LinearRing.coordinates.text
        geometry = parse_kml_coordinates(coordinates_text, 'Polygon')
        data_polygons.append({'name': name, 'geometry': geometry})

# Convert to GeoDataFrames
gdf_points = gpd.GeoDataFrame(data_points)
gdf_lines = gpd.GeoDataFrame(data_lines)
gdf_polygons = gpd.GeoDataFrame(data_polygons)

# Load the reference vector file
reference_file = r"C:\Users\91956\Desktop\Auroville Consulting\SHUBAM\shp_district\shp_district\shp_district.shp"
reference_gdf = gpd.read_file(reference_file)

# Set a buffer distance (in the same units as your coordinate system, e.g., meters)
buffer_distance = 1000

# Create a unified geometry from all reference geometries
reference_union = unary_union(reference_gdf.geometry)

# Buffer the reference geometries
buffered_reference = reference_union.buffer(buffer_distance)

# Clip and filter the geometries
clipped_points = gdf_points[gdf_points.intersects(buffered_reference)]
clipped_lines = gdf_lines[gdf_lines.intersects(buffered_reference)]
clipped_polygons = gdf_polygons[gdf_polygons.intersects(buffered_reference)]

# Save to Shapefiles
clipped_points.to_file(r"C:\Users\91956\Desktop\Auroville Consulting\SHUBAM\dams\dams3p_clipped.shp")
clipped_lines.to_file(r"C:\Users\91956\Desktop\Auroville Consulting\SHUBAM\dams\damsl_clipped.shp")
clipped_polygons.to_file(r"C:\Users\91956\Desktop\Auroville Consulting\SHUBAM\dams\dapol_clipped.shp")
